In [1]:
import os
import fileinput as fi
import glob
import moviepy.editor as mp
import pandas as pd
import random
import time


os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())


Current Working Directory  D:\02_batch_2


### Arabic Annotation collection

In [2]:
df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)

li = []

try:

    for i,dname in enumerate(df_dirname['Folder']):
        path = r'D:/02_batch_2/'
        files = glob.glob(path + dname + '/*.txt')
        print(f'Successfully created dataframe for {dname}')
        for i,f in enumerate(files):

            parse_error = False
            file_not_found = False

            stock = os.path.basename(f)            
            temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
            temp_df['filePath'] = path + dname + '/'
            temp_df['textFileName'] = stock     
            temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
            temp_df['duration'] = temp_df.stop-temp_df.start
            li.append(temp_df)
            print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

except pd.errors.ParserError:
    parse_error = True
    temp_df = pd.read_csv(f, delimiter="\t",skiprows=4)
    li.append(temp_df)
    print(f'Error for {dname} {stock} with shape {temp_df.shape}')

else:
    li.append(temp_df)


df = pd.concat(li, axis=0)
print(df.shape)
df.head(3)

In [3]:
li = []
path = r'D:/02_batch_2/'

for i,dname in enumerate(df_dirname['Folder']):

    files = glob.glob(path + dname + '/*.txt')
    #print(f'Successfully created dataframe for {dname}')
    for i,f in enumerate(files):

        parse_error = False
        file_not_found = False

        stock = os.path.basename(f)            
        temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
        temp_df['filePath'] = path + dname + '/'
        temp_df['textFileName'] = stock     
        temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
        temp_df['duration'] = temp_df.stop-temp_df.start
        li.append(temp_df)
      #  print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

df = pd.concat(li, axis=0)
print(df.shape)
df.head(2)

b'Skipping line 18: expected 5 fields, saw 7\n'
b'Skipping line 12: expected 5 fields, saw 6\n'
b'Skipping line 5: expected 5 fields, saw 8\n'


(507921, 9)


,speaker,gender,start,stop,transcript,filePath,textFileName,audioFileName,duration
0,speakeA,M,8.998978,23.522329,الحقوني الحقوني بطني بطني بموت ما أقدر الحقون...,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,14.523351
1,speakeA,M,23.522329,25.772074,أضحك عليكو بس أختبركو.,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,2.249745


In [5]:
df.duration.sum()/3600

1161.3279405906094

In [6]:
df_percent = df.groupby(['filePath','textFileName']).agg({'duration':['sum']}).reset_index()
df_percent=df_percent.droplevel(axis=1,level=1)
df_percent.rename(columns={'duration':'duration_sum'},inplace=True)

def as_perc(value, total):
    return value/float(total)

res = pd.DataFrame(df_percent.duration_sum.apply(as_perc, total=df_percent.duration_sum.sum()))
df_percent['percent'] = res

df_percent['req_duration'] = round(df_percent['percent']*180000)
df_percent.head(5)

,filePath,textFileName,duration_sum,percent,req_duration
0,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,161.231694,0.000039,7.0
1,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-1.txt,227.017000,0.000054,10.0
2,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-2.txt,100.843458,0.000024,4.0
3,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-3.txt,112.098840,0.000027,5.0
4,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-4.txt,138.146208,0.000033,6.0


In [12]:
df_percent.groupby('filePath').agg({'req_duration':'sum','percent':'sum'}).reset_index()

,filePath,req_duration,percent
0,D:/02_batch_2/37_degrees_series/,1119.0,0.006219
1,D:/02_batch_2/Almajd_kids/,4495.0,0.024986
2,D:/02_batch_2/Ba3d_altes3een_Podcast/,2741.0,0.015221
3,D:/02_batch_2/C4N/,5413.0,0.030038
4,D:/02_batch_2/Dupamicaffeine/,449.0,0.002484
5,D:/02_batch_2/Mojtm3_Ethraa/,4610.0,0.025586
6,D:/02_batch_2/Naif_aljaweni/,7453.0,0.041344
7,D:/02_batch_2/Sando9_aswad_turki_alfaisal/,1447.0,0.008040
8,D:/02_batch_2/Shabab_Ela3mal/,1491.0,0.008287
9,D:/02_batch_2/Update_NewMedia_Podcast/,831.0,0.004607


In [ ]:
for j,dname in enumerate(df_dirname['Folder']):
    
    n = random.randint(0,len(df_dirname.iloc[j,0]))
    print(len(df_dirname.iloc[j,0]))


In [ ]:
df_percent.groupby('filePath').agg({'duration_sum':'sum','percent':'sum'})

### Video Clipping

In [ ]:
hours = 180000;
duration_sum = 0;
x=0;
dest_filepath = r'D:/nf/audio/'

df_selection = pd.DataFrame(
    columns=['row_no','src_filepath','src_filepath_txt','src_filepath_wav','dest_filepath','dest_filepath_txt','dest_filepath_wav','duration','start','end']
    )

while (hours>=duration_sum):
    
    n = random.randint(0,len(df))
    fname = str(df.iloc[n,5] + df.iloc[n,7])
    duration_sum = duration_sum + df.iloc[n,8].astype('float32')

    df_selection[x,0] = n
    df_selection[x,1] = str(df.iloc[n,5])
    df_selection[x,2] = str(df.iloc[n,6])
    df_selection[x,3] = str(df.iloc[n,7])
    df_selection[x,4] = dest_filepath
    df_selection[x,5] = str(x) + "_" + str(df.iloc[n,6])
    df_selection[x,6] = str(x) + "_" + str(df.iloc[n,7])
    df_selection[x,7] = df.iloc[n,8].astype('float32')
    df_selection[x,8] = df.iloc[n,2].astype('float32')
    df_selection[x,9] = df.iloc[n,3].astype('float32')

    x=x+1  
    start = time.time()
    my_clip = mp.AudioFileClip(fname)    
    
    clip = my_clip.subclip(df.iloc[n,2],df.iloc[n,3])
    clip.write_audiofile(dest_filepath + str(x) + "_"+ str(df.iloc[n,7]) +".mp3")
    #time.sleep(1)
    end = time.time()
    

    print(duration_sum)
    print(f"Runtime of the program is {end - start}")
    # clip.write_audiofile(dest_filepath + df_selection[x,6])


df_selection.head(4) 

In [ ]:
fp = df.iloc[1,5]
fn = df.iloc[1,7]
fname = str(fp+fn)

my_clip = mp.AudioFileClip(fname)
clip = my_clip.subclip(df.iloc[1,2],df.iloc[1,3])

clip.write_audiofile(r"D:\nf\audio\1_"+str(fn)+".mp3")

#### Directory Structure

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [ ]:
list_files(r"D:\02_batch_2")